<a href="https://colab.research.google.com/github/karmanandan/NER_using_transformers/blob/main/NER_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import random
import numpy as np
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 'cuda' if torch.cuda.is_available() else 'cpu'


torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [2]:
print(f"my device ->{device}")

my device ->cpu


In [3]:
df = pd.read_csv('/content/BIOUL_data.csv')

In [4]:
df.head()

,tokens,ner_tags
0,"['Summary', '\n\n', 'Summary', 'Companies', 'T...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['The', 'U.S.', 'stock', 'market', 'has', 'suf...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['""', 'The', 'U.S.', 'yield', 'markets', '(', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Register', 'now', 'for', 'FREE', 'unlimited'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['All', '11', 'major', 'S&P', '500', '(', '.SP...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [5]:
type(df['ner_tags'][0])

str

In [6]:
#Convert str to list of str
import ast
df['ner_tags'] = df.ner_tags.apply(lambda s: list(ast.literal_eval(s)))
df['tokens'] = df.tokens.apply(lambda s: list(ast.literal_eval(s)))

In [7]:
df.head()

,tokens,ner_tags
0,"[Summary, \n\n, Summary, Companies, Tesla, dow...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[The, U.S., stock, market, has, suffered, thre...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"["", The, U.S., yield, markets, (, are, ), pull...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Register, now, for, FREE, unlimited, access, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[All, 11, major, S&P, 500, (, .SPX, ), sectors...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [8]:
type(df['ner_tags'][0])

list

In [9]:
import itertools
#Get list of NER tags from data collection notebook
ner_tags = set(itertools.chain.from_iterable(df.ner_tags))
ner_tags

{'B-CUSTOM_ORG',
 'B-CUSTOM_PERSON',
 'B-CUSTOM_PLACE',
 'B-CUSTOM_ROLE',
 'I-CUSTOM_ORG',
 'I-CUSTOM_PERSON',
 'I-CUSTOM_PLACE',
 'I-CUSTOM_ROLE',
 'L-CUSTOM_ORG',
 'L-CUSTOM_PERSON',
 'L-CUSTOM_PLACE',
 'L-CUSTOM_ROLE',
 'O',
 'U-CUSTOM_ORG',
 'U-CUSTOM_PERSON',
 'U-CUSTOM_PLACE',
 'U-CUSTOM_ROLE'}

In [10]:
len(ner_tags)

17

In [11]:
id_to_label = dict(enumerate(ner_tags))
label_to_id = {v:k for k, v in id_to_label.items()}

In [12]:
# label_to_id
dict(enumerate(ner_tags))

{0: 'I-CUSTOM_ORG',
 1: 'I-CUSTOM_PERSON',
 2: 'U-CUSTOM_ROLE',
 3: 'O',
 4: 'U-CUSTOM_PLACE',
 5: 'L-CUSTOM_ORG',
 6: 'B-CUSTOM_PERSON',
 7: 'I-CUSTOM_PLACE',
 8: 'B-CUSTOM_PLACE',
 9: 'L-CUSTOM_ROLE',
 10: 'U-CUSTOM_PERSON',
 11: 'L-CUSTOM_PERSON',
 12: 'B-CUSTOM_ORG',
 13: 'U-CUSTOM_ORG',
 14: 'B-CUSTOM_ROLE',
 15: 'L-CUSTOM_PLACE',
 16: 'I-CUSTOM_ROLE'}

In [13]:
label_to_id

{'I-CUSTOM_ORG': 0,
 'I-CUSTOM_PERSON': 1,
 'U-CUSTOM_ROLE': 2,
 'O': 3,
 'U-CUSTOM_PLACE': 4,
 'L-CUSTOM_ORG': 5,
 'B-CUSTOM_PERSON': 6,
 'I-CUSTOM_PLACE': 7,
 'B-CUSTOM_PLACE': 8,
 'L-CUSTOM_ROLE': 9,
 'U-CUSTOM_PERSON': 10,
 'L-CUSTOM_PERSON': 11,
 'B-CUSTOM_ORG': 12,
 'U-CUSTOM_ORG': 13,
 'B-CUSTOM_ROLE': 14,
 'L-CUSTOM_PLACE': 15,
 'I-CUSTOM_ROLE': 16}

In [14]:

label_to_id = {'B-CUSTOM_PLACE': 0,
 'L-CUSTOM_ORG': 1,
 'B-CUSTOM_PERSON': 2,
 'B-CUSTOM_ROLE': 3,
 'U-CUSTOM_PERSON': 4,
 'I-CUSTOM_ROLE': 5,
 'I-CUSTOM_PLACE': 6,
 'U-CUSTOM_PLACE': 7,
 'L-CUSTOM_ROLE': 8,
 'I-CUSTOM_PERSON': 9,
 'U-CUSTOM_ROLE': 10,
 'L-CUSTOM_PERSON': 11,
 'I-CUSTOM_ORG': 12,
 'U-CUSTOM_ORG': 13,
 'L-CUSTOM_PLACE': 14,
 'B-CUSTOM_ORG': 15,
 'O': 16}

In [15]:
id_to_label

{0: 'I-CUSTOM_ORG',
 1: 'I-CUSTOM_PERSON',
 2: 'U-CUSTOM_ROLE',
 3: 'O',
 4: 'U-CUSTOM_PLACE',
 5: 'L-CUSTOM_ORG',
 6: 'B-CUSTOM_PERSON',
 7: 'I-CUSTOM_PLACE',
 8: 'B-CUSTOM_PLACE',
 9: 'L-CUSTOM_ROLE',
 10: 'U-CUSTOM_PERSON',
 11: 'L-CUSTOM_PERSON',
 12: 'B-CUSTOM_ORG',
 13: 'U-CUSTOM_ORG',
 14: 'B-CUSTOM_ROLE',
 15: 'L-CUSTOM_PLACE',
 16: 'I-CUSTOM_ROLE'}

In [16]:
id_to_label = {0: 'B-CUSTOM_PLACE',
 1: 'L-CUSTOM_ORG',
 2: 'B-CUSTOM_PERSON',
 3: 'B-CUSTOM_ROLE',
 4: 'U-CUSTOM_PERSON',
 5: 'I-CUSTOM_ROLE',
 6: 'I-CUSTOM_PLACE',
 7: 'U-CUSTOM_PLACE',
 8: 'L-CUSTOM_ROLE',
 9: 'I-CUSTOM_PERSON',
 10: 'U-CUSTOM_ROLE',
 11: 'L-CUSTOM_PERSON',
 12: 'I-CUSTOM_ORG',
 13: 'U-CUSTOM_ORG',
 14: 'L-CUSTOM_PLACE',
 15: 'B-CUSTOM_ORG',
 16: 'O'}

In [17]:
def replace_fn(ls):
    new_ls = [label_to_id[key] for key in ls]
    return new_ls

In [18]:
df['ner_tags'] = df['ner_tags'].apply(replace_fn)

In [19]:
df.head()

,tokens,ner_tags
0,"[Summary, \n\n, Summary, Companies, Tesla, dow...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
1,"[The, U.S., stock, market, has, suffered, thre...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
2,"["", The, U.S., yield, markets, (, are, ), pull...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
3,"[Register, now, for, FREE, unlimited, access, ...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
4,"[All, 11, major, S&P, 500, (, .SPX, ), sectors...","[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."


In [20]:
df.iloc[0]['ner_tags']

[16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 15,
 1,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 13,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16]

In [21]:
class_counts = pd.Series(sum([item for item in df.ner_tags], [])).value_counts()

In [22]:
from collections import OrderedDict, defaultdict
class_counts_dict = class_counts.to_dict(OrderedDict)

In [23]:
class_counts_dict

OrderedDict([(16, 7305),
             (2, 89),
             (11, 89),
             (15, 52),
             (1, 52),
             (7, 48),
             (13, 46),
             (12, 32),
             (8, 17),
             (3, 17),
             (5, 10),
             (4, 8),
             (10, 8),
             (0, 7),
             (14, 7),
             (9, 7),
             (6, 3)])

In [24]:
total_count =  class_counts.sum()
total_count

7797

In [25]:
majority_sample = class_counts_dict[16]
majority_sample

7305

In [26]:
class_weights = []
for i in range(len(class_counts_dict.keys())):
    if i==16:
        class_weight = 1-(class_counts_dict[i]/total_count)
    else:
        class_weight = 1-(class_counts_dict[i]/(total_count-majority_sample))
    class_weights.append(class_weight)

In [27]:
class_weights

[0.9857723577235772,
 0.8943089430894309,
 0.8191056910569106,
 0.9654471544715447,
 0.983739837398374,
 0.9796747967479675,
 0.9939024390243902,
 0.9024390243902439,
 0.9654471544715447,
 0.9857723577235772,
 0.983739837398374,
 0.8191056910569106,
 0.9349593495934959,
 0.9065040650406504,
 0.9857723577235772,
 0.8943089430894309,
 0.06310119276644865]

In [28]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00


In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#Since the input has already been split into words, set is_split_into_words=True
tokenized_input = tokenizer(df["tokens"][0], is_split_into_words=True)

In [30]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [31]:
tokens

['[CLS]',
 'summary',
 'summary',
 'companies',
 'tesla',
 'down',
 'as',
 'q',
 '##3',
 'deliveries',
 'miss',
 'market',
 'estimates',
 'u',
 '.',
 's',
 '.',
 'factory',
 'activity',
 'slow',
 '##est',
 'in',
 '~',
 '2',
 '.',
 '5',
 'years',
 'in',
 'sept',
 '-',
 'is',
 '##m',
 'credit',
 'sui',
 '##sse',
 ',',
 'ci',
 '##ti',
 'cut',
 '202',
 '##2',
 'year',
 '-',
 'end',
 'target',
 'for',
 's',
 '&',
 'p',
 '500',
 'index',
 '##es',
 'up',
 ':',
 'dow',
 '2',
 '.',
 '66',
 '%',
 ',',
 's',
 '&',
 'p',
 '500',
 '2',
 '.',
 '59',
 '%',
 ',',
 'nas',
 '##da',
 '##q',
 '2',
 '.',
 '27',
 '%',
 'oct',
 '3',
 '(',
 'reuters',
 ')',
 '-',
 'wall',
 'street',
 "'",
 's',
 'three',
 'major',
 'index',
 '##es',
 'rallied',
 'to',
 'close',
 'over',
 '2',
 '%',
 'on',
 'monday',
 'as',
 'u',
 '.',
 's',
 '.',
 'treasury',
 'yields',
 'tumbled',
 'on',
 'weaker',
 '-',
 'than',
 '-',
 'expected',
 'manufacturing',
 'data',
 ',',
 'increasing',
 'the',
 'appeal',
 'of',
 'stocks',
 'at',
 '

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#Since the input has already been split into words, set is_split_into_words=True
tokenized_input = tokenizer(['I','am','Elon'], is_split_into_words=True)
print("token_ids:",tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print("tokens:",tokens)
print("word_ids:",tokenized_input.word_ids())# index of orifginal word

token_ids: {'input_ids': [101, 1045, 2572, 3449, 2239, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}
tokens: ['[CLS]', 'i', 'am', 'el', '##on', '[SEP]']
word_ids: [None, 0, 1, 2, 2, None]


In [33]:
X = list(df['tokens'])
y = list(df['ner_tags'])

In [34]:
def FindMaxLength(lst):
    maxList = max(lst, key = lambda i: len(i))
    maxLength = len(maxList)
     
    return maxLength

In [35]:
FindMaxLength(X)

103

In [36]:
#We have maximum tokens as 103 but Let's assume max_length as 200
max_length = 200

In [37]:
from transformers import AutoTokenizer

class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y,max_length,id_to_label):
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        self.X = X
        self.y = y
        self.id_to_label = id_to_label
    
    def __getitem__(self, idx):
        return self.tokenize_and_align_labels(self.X[idx],self.y[idx])

    def __len__(self):
        return len( self.X)
    
    def tokenize_and_align_labels(self,x_el,y_el):
        #print(row["tokens"])
        #print( row['ner_tags'])
        tokenized_inputs = self.tokenizer(x_el, truncation=True,padding="max_length", is_split_into_words=True,max_length=max_length)
        data = {key: torch.tensor(val) for key, val in tokenized_inputs.items()}
        #print(tokenized_inputs)
        ner_tags = y_el
        labels = []
        word_ids = tokenized_inputs.word_ids(batch_index=0)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(ner_tags[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

        data["labels"] = torch.tensor(labels).squeeze()
        return data

train_dataset = Dataset(X, y,max_length,id_to_label)

In [38]:
for data in train_dataset:
    print(data['input_ids'])
    print(data['labels'])
    break

tensor([  101, 12654, 12654,  3316, 26060,  2091,  2004,  1053,  2509, 23534,
         3335,  3006, 10035,  1057,  1012,  1055,  1012,  4713,  4023,  4030,
         4355,  1999,  1066,  1016,  1012,  1019,  2086,  1999, 17419,  1011,
         2003,  2213,  4923, 24086, 11393,  1010, 25022,  3775,  3013, 16798,
         2475,  2095,  1011,  2203,  4539,  2005,  1055,  1004,  1052,  3156,
         5950,  2229,  2039,  1024, 23268,  1016,  1012,  5764,  1003,  1010,
         1055,  1004,  1052,  3156,  1016,  1012,  5354,  1003,  1010, 17235,
         2850,  4160,  1016,  1012,  2676,  1003, 13323,  1017,  1006, 26665,
         1007,  1011,  2813,  2395,  1005,  1055,  2093,  2350,  5950,  2229,
        24356,  2000,  2485,  2058,  1016,  1003,  2006,  6928,  2004,  1057,
         1012,  1055,  1012,  9837, 16189, 18303,  2006, 15863,  1011,  2084,
         1011,  3517,  5814,  2951,  1010,  4852,  1996,  5574,  1997, 15768,
         2012,  1996,  2707,  1997,  1996,  2095,  1005,  1055, 

In [39]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=17)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [40]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    report_to =None

)

#We should override comput_loss to inform trainerr about class imbalance(we have majority as "O" i.e not an entity)
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 17 labels with different weights)
        #class_weights = torch.tensor([3.0, 3.0, 3.0,3.0, 3.0, 3.0,3.0, 3.0, 3.0,3.0, 3.0, 3.0,3.0, 3.0, 3.0,3.0, 0.2])
        class_weights = torch.tensor([0.9857723577235772,0.8943089430894309, 0.8191056910569106, 0.9654471544715447, 0.983739837398374,
                        0.9796747967479675, 0.9939024390243902, 0.9024390243902439, 0.9654471544715447, 0.9857723577235772, 0.983739837398374, 0.8191056910569106,
                        0.9349593495934959, 0.9065040650406504, 0.9857723577235772, 0.8943089430894309, 0.06310119276644865])
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights,reduction='mean')
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

In [41]:
os.environ["WANDB_DISABLED"] = "true"

In [42]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=32, training_loss=1.543299913406372, metrics={'train_runtime': 249.9291, 'train_samples_per_second': 1.0, 'train_steps_per_second': 0.128, 'total_flos': 12762552300000.0, 'train_loss': 1.543299913406372, 'epoch': 1.0})

In [43]:
trainer.save_model("custom_ner_transformers")

In [44]:
#Inference Code
from transformers import AutoModelForTokenClassification,AutoTokenizer
MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
saved_model = AutoModelForTokenClassification.from_pretrained("/content/custom_ner_transformers")

In [45]:
import spacy.cli

spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [72]:
#Inference Code
import spacy
nlp = spacy.load("en_core_web_lg")
# text = 'The Tesla factory is seen in Fremont, California, U.S'
# text = 'My name is Elon Musk and I am CEO of Tesla'
text = 'Tim Cook says Apple to add AI to products on thoughtful basis'
spacy_tokens = nlp(text)
text_tokens = [token.text for token in spacy_tokens]
tokenized_inputs = tokenizer(text_tokens,truncation=True,padding="max_length", is_split_into_words=True,max_length=200, return_tensors="pt")
outputs = saved_model(**tokenized_inputs)

In [73]:
outputs.logits.squeeze()[0]

tensor([-0.6387,  0.0798,  0.1659,  0.0435, -0.4011, -0.8006, -0.9003,  0.2698,
        -0.6972, -0.7931, -1.0744,  0.1567, -1.0766, -0.2261, -1.1851, -0.2103,
         3.8922], grad_fn=<SelectBackward0>)

In [74]:
torch.argmax(outputs.logits.squeeze()[0])

tensor(16)

In [75]:
predictions = torch.argmax(outputs.logits.squeeze(), axis=1)
print(predictions.shape)
predictions = [id_to_label[int(i)] for i in predictions]
print(predictions)
words = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0])
print(words)
word_ids = tokenized_inputs.word_ids()
print("word_ids:",word_ids)# index of original word
print("words:",text_tokens)

torch.Size([200])
['O', 'B-CUSTOM_PERSON', 'L-CUSTOM_PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [76]:
#Map each token to it's original word and create map of word-entity type
token_entity = {}
for idx,word in enumerate(text_tokens):
    print(idx,word)
    for id,word_id in enumerate(word_ids):
        if idx==word_id:
            if word not in token_entity:
                token_entity[word] = predictions[id]

0 Tim
1 Cook
2 says
3 Apple
4 to
5 add
6 AI
7 to
8 products
9 on
10 thoughtful
11 basis


In [77]:
token_entity

{'Tim': 'B-CUSTOM_PERSON',
 'Cook': 'L-CUSTOM_PERSON',
 'says': 'O',
 'Apple': 'O',
 'to': 'O',
 'add': 'O',
 'AI': 'O',
 'products': 'O',
 'on': 'O',
 'thoughtful': 'O',
 'basis': 'O'}

In [78]:
for token in spacy_tokens:
  print(token.text, token.idx, token.idx + len(token.text))

Tim 0 3
Cook 4 8
says 9 13
Apple 14 19
to 20 22
add 23 26
AI 27 29
to 30 32
products 33 41
on 42 44
thoughtful 45 55
basis 56 61


In [79]:
output = []
for token in spacy_tokens:
    if token_entity[token.text] !='O':
        output.append(
                {
                    "start": token.idx,
                    "end": token.idx + len(token.text),
                    "label":token_entity[token.text],
                }
            )

In [80]:
output

[{'start': 0, 'end': 3, 'label': 'B-CUSTOM_PERSON'},
 {'start': 4, 'end': 8, 'label': 'L-CUSTOM_PERSON'}]

In [81]:
ex = [{"text": text, "ents": output}]
spacy.displacy.render(
                ex, style="ent", manual=True
            )

'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Tim\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">B-CUSTOM_PERSON</span>\n</mark>\n \n<mark class="entity" style="background: #ddd; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Cook\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">L-CUSTOM_PERSON</span>\n</mark>\n says Apple to add AI to products on thoughtful basis</div>'